# Frontend 구현 및 배포

2주차_코멘토_인공지능_Backend_이름.ipynb를 작성해 주셨으면, 이를 바탕으로 실제로 작동하는 프로토타입을 완성할 수 있는 예제코드입니다.

# 1. 과제 안내

- PDF 문서 분석 및 질의응답: PDF 문서를 업로드하면, 문서 내용을 분석하여 관련 질문에 대한 답변을 제공하는 RAG 파이프라인을 Frontend와 연동하는 과정입니다.

- Streamlit 웹 인터페이스: Streamlit을 사용하여 사용자가 쉽게 접근하고 질문할 수 있는 직관적인 웹 기반 챗봇 UI를 제공합니다.

- API 기반 모델 활용: 최신모델을 사용하여 정확하고 상세한 답변을 생성하며, 질문과 관련된 추가 질문까지 제안하여 사용자 경험을 향상시켜 보세요.

- 답변 요약 및 시각화: 생성된 답변을 요약하여 제공하며, 검색된 문서의 주요 키워드를 워드 클라우드로 시각화하여 한눈에 내용을 파악할 수 있도록 돕습니다.

- PDF 답변 다운로드: 생성된 답변을 PDF 파일로 다운로드할 수 있는 기능을 제공하여 정보 활용성을 높였습니다.

- 로컬 환경 실행 지원: localtunnel을 통해 로컬 환경에서도 쉽게 챗봇에 접속하고 테스트할 수 있습니다.

# 2. 예제 코드 활용 및 테스트 방법
- Colab에서 코드를 실행합니다.
- 생성된 Streamlit 애플리케이션 링크에 접속합니다.
- 사이드바에 있는 "PDF 파일 업로드" 버튼을 클릭하여 ETF 관련 PDF 문서를 업로드합니다.
- 하단의 입력창에 ETF 관련 질문을 입력하면 챗봇이 문서 내용을 기반으로 답변을 생성해줍니다.

In [1]:
!pip install -q streamlit
!pip install reportlab

In [2]:
!pip install python-dotenv
!pip install faiss-cpu
!pip install pymupdf

In [7]:
!pip install langchain_community
!pip install langchain_openai
!pip install wordcloud
!pip install matplotlib

In [ ]:
# Streamlit 앱 코드 작성 및 저장
# 2차, 3차 업무를 통해 구현한 RAG 파이프라인을 활용해서 앱 코드를 작성해주세요.
%%writefile app.py

import streamlit as st
from dotenv import load_dotenv
import openai
import os
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.chains.summarize import load_summarize_chain
from io import BytesIO
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
import base64
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Streamlit 페이지 설정
st.set_page_config(page_title="ETF 질의응답 챗봇", page_icon=":robot:")

# 제목 및 설명
st.title("ETF 질의응답 챗봇")
st.markdown("ETF 관련 질문을 입력하고 PDF 문서를 업로드하세요.")

# 세션 상태 초기화
if "messages" not in st.session_state:
    st.session_state.messages = []

# 챗 히스토리 표시
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# 파일 업로드 위젯 (사이드바)
with st.sidebar:
    uploaded_file = st.file_uploader("PDF 파일 업로드", type="pdf") #pdf 외에도 다양한 형식으로 시도해 볼 수 있습니다.

# 환경 변수 로드 및 OpenAI API 키 설정 (숨김)
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
openai.api_key = api_key

# 여기서부터 2차,3차 업무를 거쳐 구현한 RAG 파이프라인입니다.
if uploaded_file and api_key:
    # 파일을 임시 파일로 저장
    with open("temp.pdf", "wb") as f:
        f.write(uploaded_file.read())

    # 문서 로드
    loader = PyMuPDFLoader("temp.pdf")
    docs = loader.load()

    # 문서 미리보기
    st.sidebar.subheader("문서 미리보기")
    st.sidebar.write(docs[0].page_content[:500] + "...")

    # 문서 분할
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    split_documents = text_splitter.split_documents(docs)

    # 임베딩 생성
    embeddings = OpenAIEmbeddings()

    # 벡터 DB 생성 및 저장
    vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

    # 검색기 생성
    retriever = vectorstore.as_retriever()

    # 프롬프트 템플릿
    prompt = PromptTemplate.from_template(
        """
        너는 ETF 투자 관련 정보를 제공하는 전문가야.
        다음과 같이 세부 사항을 포함해서 제시해.
        1. 참조한 가이드라인 또는 보고서의 출처 및 페이지 정보
        2. 받은 질문과 유사한 투자자들이 관심 가질만한 질문 3가지

        질문:
        {question}

        검색된 문서:
        {context}

        답변 예시:
        질문: 특정 산업에 투자하는 ETF를 찾고 있는데, 어떤 기준으로 선택해야 하나요?
        답변: 특정 산업 ETF 선택 시에는 다음 사항을 고려해야 합니다.
        1. 기초지수의 구성 종목 및 비중 확인
        2. 운용보수 및 총보수 비용 비교
        3. 추적 오차 및 괴리율 분석
        4. 거래량 및 유동성 확인
        5. 장기적인 산업 전망 및 성장 가능성 평가

        출처:
        참조 보고서: ETF 투자 가이드라인
        페이지: 12페이지

        유사한 질문
        "특정 국가의 주식 시장에 투자하는 ETF를 추천해주세요."
        "배당 수익률이 높은 ETF에는 어떤 것들이 있나요?"
        "최근 주목받는 테마형 ETF는 무엇인가요?"
        """
    )

    # LLM 생성
    llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

    # 체인 생성
    chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

    # 답변 요약 체인 생성
    summarize_chain = load_summarize_chain(llm, chain_type="stuff")

# 여기서부터 Streamlit을 활용한 프론트엔드 구현 부분입니다. 자신이 기획한 서비스에 맞게 다양한 기능들을 추가해 보세요.
# 질의응답 입력 위젯
if prompt := st.chat_input("질문을 입력하세요:"):
        st.session_state.messages.append({"role": "user", "content": prompt})
        st.chat_message("user").markdown(prompt)

        # 체인 실행 및 결과 출력
        with st.spinner("답변 생성 중..."):
            response = chain.invoke(prompt)

            # 답변 요약
            summary = summarize_chain.run(response)

            # 답변 및 요약 표시
            st.session_state.messages.append({"role": "assistant", "content": f"**요약:** {summary}\n\n**전체 답변:**\n{response}"})
            st.chat_message("assistant").markdown(f"**요약:** {summary}\n\n**전체 답변:**\n{response}")

            # 답변 다운로드 기능
            pdf_buffer = BytesIO()
            c = canvas.Canvas(pdf_buffer, pagesize=letter)
            c.drawString(100, 750, response)
            c.save()
            pdf_out = pdf_buffer.getvalue()

            st.download_button(
                label="답변 다운로드 (PDF)",
                data=pdf_out,
                file_name="response.pdf",
                mime="application/pdf"
            )

            # 검색 결과 시각화 (워드 클라우드)
            text = " ".join([doc.page_content for doc in retriever.get_relevant_documents(prompt)])
            wordcloud = WordCloud(width=800, height=400, background_color="white").generate(text)
            plt.figure(figsize=(10, 5))
            plt.imshow(wordcloud, interpolation="bilinear")
            plt.axis("off")
            st.pyplot(plt)

            # 사용자 맞춤 설정 (답변 길이)
            if st.checkbox("짧은 답변 보기"):
                st.write(f"**짧은 답변:** {summary}")
else:
    st.info("사이드바에서 PDF 파일을 업로드해주세요.")

2025-07-09 22:00:33.979 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-09 22:00:33.980 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-09 22:00:33.996 
  command:

    streamlit run /Users/choiwonjun/Project/edu-genai-platform/.venv/lib/python3.13/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-07-09 22:00:33.997 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-09 22:00:33.997 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-09 22:00:33.997 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-09 22:00:33.998 Thread 'MainThread': missing ScriptRunContext! 

In [9]:
# localtunnel을 사용하기 전에 사용자의 공용 IP 주소를 확인합니다.
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

# "Password/Enpoint IP for localtunnel is:" 우측에 표시되는 숫자를 복사하세요

Password/Enpoint IP for localtunnel is: 58.225.184.166


In [11]:
#Node.js 패키지 관리자인 npm(Node Package Manager)을 사용합니다.
!npm install localtunnel


⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇
added 22 packages in 2s
⠇
⠇3 packages are looking for funding
⠇  run `npm fund` for details
⠇

In [19]:
#Streamlit 앱을 백그라운드에서 실행하고, 실행 로그를 파일에 저장합니다.
# !streamlit run app.py &>/content/logs.txt &
!python -m streamlit run app.py



      👋 Welcome to Streamlit!

      If you'd like to receive helpful onboarding emails, news, offers, promotions,
      and the occasional swag, please enter your email address below. Otherwise,
      leave this field blank.

      Email:  ^C


In [18]:
#localtunnel을 사용하여 로컬 서버의 8501 포트를 외부에서 접근할 수 있도록 터널링합니다.
!npx localtunnel --port 8501

# "your url is:" 우측에 표시되는 url 링크를 클릭한 뒤에, 위에서 복사한 숫자를 붙혀넣으면 app으로 이동합니다.


⠙your url is: https://three-ends-behave.loca.lt


OSError: [Errno 5] Input/output error